# Scraping Country Population Data for COVID Dashboard

- Want to normalize statistics according to population
- Plan is to scrape the table from: 
    - https://www.worldometers.info/world-population/population-by-country/ 

- Actually, nevermind. Will use Kaggle:
    - https://www.kaggle.com/tanuprabhu/population-by-country-2020?select=population_by_country_2020.csv

In [12]:
!pip install -U fsds
from fsds.imports import *

fsds v0.2.27 loaded.  Read the docs: https://fs-ds.readthedocs.io/en/latest/ 


Handle,Package,Description
dp,IPython.display,Display modules with helpful display and clearing commands.
fs,fsds,Custom data science bootcamp student package
mpl,matplotlib,Matplotlib's base OOP module with formatting artists
plt,matplotlib.pyplot,Matplotlib's matlab-like plotting module
np,numpy,scientific computing with Python
pd,pandas,High performance data structures and tools
sns,seaborn,High-level data visualization library based on matplotlib


[i] Pandas .iplot() method activated.


In [13]:
import functions as fn

In [14]:
# import requests, bs4

# r = requests.get('https://www.worldometers.info/world-population/population-by-country/ ')
# soup = bs4.BeautifulSoup(r.content)

In [15]:
# len(soup.find_all('table'))

In [16]:
# soup.find_all('tbody')

## Using Kaggle Dataset

In [17]:
pop = pd.read_csv('Reference Data/population_by_country_2020.csv')
pop

,Country (or dependency),Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
0,China,1440297825,0.39 %,5540090,153,9388211,-348399.0,1.7,38,61 %,18.47 %
1,India,1382345085,0.99 %,13586631,464,2973190,-532687.0,2.2,28,35 %,17.70 %
2,United States,331341050,0.59 %,1937734,36,9147420,954806.0,1.8,38,83 %,4.25 %
3,Indonesia,274021604,1.07 %,2898047,151,1811570,-98955.0,2.3,30,56 %,3.51 %
4,Pakistan,221612785,2.00 %,4327022,287,770880,-233379.0,3.6,23,35 %,2.83 %
...,...,...,...,...,...,...,...,...,...,...,...
230,Montserrat,4993,0.06 %,3,50,100,NaN,N.A.,N.A.,10 %,0.00 %
231,Falkland Islands,3497,3.05 %,103,0,12170,NaN,N.A.,N.A.,66 %,0.00 %
232,Niue,1628,0.68 %,11,6,260,NaN,N.A.,N.A.,46 %,0.00 %
233,Tokelau,1360,1.27 %,17,136,10,NaN,N.A.,N.A.,0 %,0.00 %


In [20]:
corona = fn.CoronaData(verbose=True,run_workflow=True)
df_world=corona.df

[i] DOWNLOADING DATA USING KAGGLE API
	https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset
	- Downloaded dataset .zip and extracted to:"New Data/"
	- Extraction Complete.


,Date,Province/State,Country/Region,Confirmed,Deaths,Recovered
0,2020-01-22,Anhui,Mainland China,1.0,0.0,0.0
1,2020-01-22,Beijing,Mainland China,14.0,0.0,0.0
2,2020-01-22,Chongqing,Mainland China,6.0,0.0,0.0
3,2020-01-22,Fujian,Mainland China,1.0,0.0,0.0
4,2020-01-22,Gansu,Mainland China,0.0,0.0,0.0


[i] There are 223 countries in the datatset
[i] Dates Covered:
	From 01-22-2020 to 09-23-2020


In [21]:
## Get WORLD dictionary with all countries
grouping_col = 'Country/Region'
countries = list(df_world.groupby(grouping_col).groups.keys())

WORLD = {}
for country in countries:
#     print(country)
    WORLD[country] = fn.get_group_ts(df_world,country, grouping_col)

In [24]:
WORLD['Russia']

,Russia - Confirmed,Russia - Deaths,Russia - Recovered
Date,,,
2020-01-31,2.0,0.0,0.0
2020-02-01,2.0,0.0,0.0
2020-02-02,2.0,0.0,0.0
2020-02-03,2.0,0.0,0.0
2020-02-04,2.0,0.0,0.0
...,...,...,...
2020-09-19,1092915.0,19270.0,903545.0
2020-09-20,1098958.0,19349.0,906431.0
2020-09-21,1105048.0,19420.0,909026.0


In [27]:
pop[pop['Country (or dependency)']=='Russia']

,Country (or dependency),Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
8,Russia,145945524,0.04 %,62206,9,16376870,182456.0,1.8,40,74 %,1.87 %


In [30]:
pop = pop[['Country (or dependency)','Population (2020)']]
pop.columns= ['country','population']
pop

,country,population
0,China,1440297825
1,India,1382345085
2,United States,331341050
3,Indonesia,274021604
4,Pakistan,221612785
...,...,...
230,Montserrat,4993
231,Falkland Islands,3497
232,Niue,1628
233,Tokelau,1360


In [36]:
countries_world = list(WORLD.keys())
countries_pop = pop['country'].unique().tolist()
len(countries_pop),len(countries_world)

(235, 223)

In [42]:
## Countries in Covid Data that are NOT in population data
unmatched_countries = list(filter(lambda x: x not in countries_pop,countries_world))
unmatched_countries

[' Azerbaijan',
 "('St. Martin',)",
 'Bahamas, The',
 'Burma',
 'Cape Verde',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 'Curacao',
 'Czech Republic',
 'Diamond Princess',
 'East Timor',
 'Faroe Islands',
 'Gambia, The',
 'Guernsey',
 'Ivory Coast',
 'Jersey',
 'Kosovo',
 'MS Zaandam',
 'Macau',
 'Mainland China',
 'North Ireland',
 'Others',
 'Palestine',
 'Republic of Ireland',
 'Republic of the Congo',
 'Reunion',
 'Saint Kitts and Nevis',
 'Saint Vincent and the Grenadines',
 'Sao Tome and Principe',
 'St. Martin',
 'The Bahamas',
 'The Gambia',
 'UK',
 'US',
 'Vatican City',
 'West Bank and Gaza',
 'occupied Palestinian territory']

In [41]:
unmatched_populaton_countries =list(filter(lambda x: x not in countries_world,countries_pop))
unmatched_populaton_countries

['China',
 'United States',
 'DR Congo',
 'United Kingdom',
 'Myanmar',
 "Côte d'Ivoire",
 'North Korea',
 'Czech Republic (Czechia)',
 'Turkmenistan',
 'Congo',
 'State of Palestine',
 'Réunion',
 'Solomon Islands',
 'Macao',
 'Vanuatu',
 'New Caledonia',
 'French Polynesia',
 'Sao Tome & Principe',
 'Samoa',
 'Curaçao',
 'Kiribati',
 'Micronesia',
 'St. Vincent & Grenadines',
 'Tonga',
 'U.S. Virgin Islands',
 'Isle of Man',
 'Bermuda',
 'Marshall Islands',
 'Northern Mariana Islands',
 'American Samoa',
 'Saint Kitts & Nevis',
 'Faeroe Islands',
 'Sint Maarten',
 'Turks and Caicos',
 'Saint Martin',
 'British Virgin Islands',
 'Caribbean Netherlands',
 'Palau',
 'Cook Islands',
 'Anguilla',
 'Tuvalu',
 'Wallis & Futuna',
 'Nauru',
 'Saint Helena',
 'Saint Pierre & Miquelon',
 'Montserrat',
 'Falkland Islands',
 'Niue',
 'Tokelau']

In [ ]:
rename_dict = {''}